In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob
import scipy
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint


In [ ]:
os.chdir(r'/content/drive/MyDrive/test')

In [ ]:
os.chdir(r'/content/drive/MyDrive/test')
full_list = glob.glob(r'./train_set/full/*.jpg')
free_list = glob.glob(r'./train_set/empty/*.jpg')

full_img = img_to_array(load_img(full_list[3]), dtype=np.uint8)
free_img = img_to_array(load_img(free_list[4]), dtype=np.uint8)

fig, ax = plt.subplots(1,2, figsize=(16,16))
ax = ax.ravel()

ax[0].set_title(len(full_list))
ax[0].imshow(full_img)

ax[1].set_title(len(free_list))
ax[1].imshow(free_img)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    shear_range=0.01,
    zoom_range=[0.9, 1.1],
    validation_split=0.1,
    preprocessing_function=preprocess_input
)

val_datagen = ImageDataGenerator(
    validation_split=0.1,
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    r'./train_set',
    target_size=(48, 48),
    classes=['full', 'empty'],
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    subset='training'
)

val_gen = val_datagen.flow_from_directory(
    r'./train_set',
    target_size=(48, 48),
    classes=['full', 'empty'],
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    subset='validation'
)

print(val_gen.class_indices)

In [ ]:
base_model = MobileNetV2(input_shape=(48, 48, 3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = True   #layer의 weight를 지속적으로 변하게 하기 위해해

In [ ]:
history = model.fit_generator(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[
        ModelCheckpoint('model.h6', monitor='val_acc', save_best_only=True, verbose=1)
    ]
)

In [ ]:
model = load_model('model.h6')  #저장한 모델을 다시 로드

last_weight = model.layers[-1].get_weights()[0] # (1280, 2)
#activation map을 그려야 하므로 모델의 마지막 layer의odel = load_model('model.h5')


new_model = Model(
    inputs=model.input,
    outputs=(
        model.layers[-3].output, # the layer just before GAP, for using spatial features
        model.layers[-1].output
    )
)

new_model.summary()

In [ ]:
'''
'./data/Test/e2.jpg'
 full_list[]
 free_list[]
'''

os.chdir(r'/content/drive/MyDrive/test')
test_img = img_to_array(load_img('/content/drive/MyDrive/test/test_set/test3.jpg', target_size=(48, 48)))

test_input = preprocess_input(np.expand_dims(test_img.copy(), axis=0))

pred = model.predict(test_input)
print(pred)
plt.figure(figsize=(8, 8))
plt.title('%.2f%% Free' % (pred[0][1] * 100))
plt.imshow(test_img.astype(np.uint8))